In [1]:
import warnings
warnings.filterwarnings('ignore')
import os, gc
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,GroupKFold
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error

from catboost import CatBoostRegressor,Pool

from tqdm.notebook import tqdm

In [2]:
root_path = "../../../input/"
train = np.load("../../data_preprocessing/new_cite_train_final.npz")["arr_0"]
target = pd.read_hdf(f"{root_path}open-problems-multimodal/train_cite_targets.h5").values
target -= target.mean(axis=1).reshape(-1, 1)
target /= target.std(axis=1).reshape(-1, 1)
print(train.shape,target.shape)

(70988, 735) (70988, 140)


In [4]:
train_index = np.load(f"{root_path}/multimodal-single-cell-as-sparse-matrix/train_cite_inputs_idxcol.npz",allow_pickle=True)
meta = pd.read_csv(f"{root_path}open-problems-multimodal/metadata.csv",index_col = "cell_id")
meta = meta[meta.technology=="citeseq"]
lbe = LabelEncoder()
meta["cell_type"] = lbe.fit_transform(meta["cell_type"])
meta["gender"] = meta.apply(lambda x:0 if x["donor"]==13176 else 1,axis =1)
meta_train = meta.reindex(train_index["index"])
train_meta = meta_train["gender"].values.reshape(-1, 1)
train = np.concatenate([train,train_meta],axis= -1)
train_meta = meta_train["cell_type"].values.reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
train_meta = ohe.fit_transform(train_meta)
train = np.concatenate([train,train_meta],axis= -1)
train.shape

(70988, 743)

In [5]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)


In [6]:
# MOCB
class MultiOutputCatboostRegressor:
    def __init__(self,params):
        self.params = params
        self.model_list = []

    def fit(self,train_data,train_label,val_data,val_label,**fit_params):
        
        output_num = train_label.shape[1]
        for i in tqdm(range(output_num)):
            model = CatBoostRegressor(**self.params)
            train_pool = Pool(train_data,train_label[:,i])
            model.fit(train_pool,
                eval_set = (val_data,val_label[:,i]),
                early_stopping_rounds = 10,
                verbose_eval = 0,
                use_best_model = True,
                )
            self.model_list.append(model)
            
    def predict(self,test_data):
        test_data = Pool(test_data)
        res_list = []
        for model in tqdm(self.model_list):
            res = model.predict(test_data,thread_count = 19)
            res_list.append(res)
        res_list = np.stack(res_list,axis = 1)
        return res_list
        
    def dump(self,path = "./models/MOCB/" ):
        count = 0
        os.makedirs(path,exist_ok=True)
        for model in tqdm(self.model_list):
            model_path = f'{path}model_{str(count)}.cbm'
            model.save_model(model_path)
            count += 1
        print("Model saved")

    def load(self,path = "./models/MOCB/" ):
        models = os.listdir(path)
        if len(self.model_list) != 0:
            raise ValueError("Don't load! Already loaded!")
        else:
            for i in tqdm(range(len(models))):
                # print("begin")
                model = CatBoostRegressor(self.params)
                model_path = f'{path}model_{i}.cbm'
                model.load_model(model_path)
                # print("end")
                self.model_list.append(model)
            print("Model loaded")

In [7]:
params = {'learning_rate': 0.004, 
          'depth': 6, 
          'l2_leaf_reg': 9.84065, 
          'loss_function': 'RMSE', 
          'eval_metric': 'RMSE', 
          'task_type': 'GPU', 
          'iterations': 10000,
          'od_type': 'Iter', 
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bayesian', 
          'allow_const_label': True, 
          'random_state': 1,
          "bagging_temperature":2,
         }


In [8]:
%%time
np.random.seed(42)
kf = GroupKFold(n_splits=3) 
score = []

# model = Ridge(copy_X=False)
print('Train...')
for id,(idx_tr, idx_va) in enumerate(kf.split(range(train.shape[0]),groups= meta_train.donor)):
    Xtr, Xva = train[idx_tr], train[idx_va]
    Ytr, Yva = target[idx_tr], target[idx_va]
    print(f'Fold {id}..')
    model = MultiOutputCatboostRegressor(params)
    model.fit(Xtr,Ytr,Xva,Yva)
    
    y_tr_pred = model.predict(Xtr)
    mse_tr = mean_squared_error(Ytr, y_tr_pred)
    mae_tr = mean_absolute_error(Ytr, y_tr_pred)
    pearson_tr = correlation_score(Ytr, y_tr_pred)
    print(f"Flod_{id}_train  mse:{mse_tr},  mae:{mae_tr},  pearson:{pearson_tr}")

    y_va_pred = model.predict(Xva)
    mse = mean_squared_error(Yva, y_va_pred)
    mae = mean_absolute_error(Yva, y_va_pred)
    pearson = correlation_score(Yva, y_va_pred)
    print(f"Flod-{id}_test   mse:{mse},  mae:{mae},  pearson:{pearson}\n")

    score.append(pearson)
    del Xtr, Ytr
    del Xva, Yva
    gc.collect()
    
    filename = f"./models/MOCB/fold{id}/"
    os.makedirs(filename,exist_ok=True)
    model.dump(filename)
gc.collect()

Train...
Fold 0..


  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

Flod_0_train  mse:0.1789837632098236,  mae:0.2921363536284119,  pearson:0.9060354528017331


  0%|          | 0/140 [00:00<?, ?it/s]

...

  0%|          | 0/140 [00:00<?, ?it/s]

Flod-2_test   mse:0.20754699602146018,  mae:0.31563993192721085,  pearson:0.8899201197327345



  0%|          | 0/140 [00:00<?, ?it/s]

Model saved
CPU times: total: 1h 30min 36s
Wall time: 2h 45min 18s


27

## Predict

In [12]:
test = np.load("../../data_preprocessing/new_cite_test_final.npz")["arr_0"]

test_index = np.load(f"{root_path}/multimodal-single-cell-as-sparse-matrix/test_cite_inputs_idxcol.npz",allow_pickle=True)
meta_test = meta.reindex(test_index["index"])
test_meta = meta_test["gender"].values.reshape(-1, 1)
test = np.concatenate([test,test_meta],axis= -1)
test_meta = meta_test["cell_type"].values.reshape(-1, 1)
test_meta = ohe.transform(test_meta)
test = np.concatenate([test,test_meta],axis= -1)
test.shape

(48663, 743)

In [13]:
def std(x):
    return (x - np.mean(x,axis=1).reshape(-1,1)) / np.std(x,axis=1).reshape(-1,1)

In [14]:
score

[0.890279763295954, 0.8956999948811826, 0.8899201197327345]

In [15]:
from tqdm.notebook import tqdm
import glob
model_path = "./models/MOCB/fold*"
model_list = glob.glob(model_path)
preds = np.zeros((test.shape[0], 140))
for id,fn in enumerate(tqdm(model_list)):
    model_ = MultiOutputCatboostRegressor(params)
    model_.load(fn+"/")
    preds += std(model_.predict(test))* score[id]
    gc.collect()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

Model loaded


  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

Model loaded


  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

Model loaded


  0%|          | 0/140 [00:00<?, ?it/s]

In [16]:
def submit(test_pred,multi_path):
    submission = pd.read_csv(multi_path,index_col = 0)
    submission = submission["target"]
    print("data loaded")
    submission.iloc[:len(test_pred.ravel())] = test_pred.ravel()
    assert not submission.isna().any()
    # submission = submission.round(6) # reduce the size of the csv
    print("start -> submission.zip")
    submission.to_csv('submission.zip')
    print("submission.zip saved!")

In [17]:
%%time
submit(preds,multi_path = r"D:\python_project\MSCI\model_ensemble\submission_best.zip")

data loaded
start -> submission.zip
submission.zip saved!
CPU times: total: 1min 22s
Wall time: 3min 21s
